# Web Scraping con Scrapy

## Introducción

El Web Scraping es una técnica para extraer información de la web de manera automática cuando no se tiene una API que la proporcione.

### Ventajas
- No se depende de una API
- No hay limitaciones (tiempo, información)

### Desventajas
- Se depende de la estructura de la página a la cual se realiza el scraping

### Proceso
1. Se obtiene una url semilla
2. Se realizan las solicitudes (Request)
3. Se obtiene una respuesta (Response)
4. Se obtiene la información de los items (Populate Items)
5. Ir a más URLs y se repite el proceso.

## Tipos de Web Scraping

- Una sola página web  
Se emplea un Spyder (*scrapy.spiders.Spyder*)
  
- Varías páginas web  
Se emplrea un Crawler (*scrapy.spiders.CrawlSpider*). Se compone del **crawling vertical** y **crawling horizontal**.

### Crawling vertical
Es el barrido de información que se hace para cada uno de los items en una página (se accede a la página que es propia del item).

### Crawling horizontal
Es el barrido de información que se realiza para múltiples páginas de un mismo dominio (paginación) en donde están distribuidos los items.

## Spider

In [ ]:
from scrapy.item import Field, Item
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.loader import ItemLoader


class Pregunta(Item):
    """Clase que define los Items
    id: corresponde al id secuencial de las preguntas
    pregunta: corresponde a la pregunta de StackOverflow
    """
    id = Field()
    pregunta = Field()

    
class StackOverflowSpider(Spider):
    name = "Spider_01"  # Nombre del Spider
    start_urls = ['https://es.stackoverflow.com/questions']  # url semilla

    def parse(seld, response):
        selector = Selector(response)
        # // -> Indica la raíz del arbol (padre)
        # //*[@id="questions"] -> Div con todas las preguntas
        preguntas = selector.xpath('//div[@id="questions"]/div')
        # Se itera sobre las preguntas
        for n, pregunta in enumerate(preguntas):
            # Creamos el Item
            i = ItemLoader(Pregunta(), pregunta)
            # //*[@id="question-summary-189320"]/div[2]/h3/a
            i.add_xpath('pregunta', './/h3/a[@class="question-hyperlink"]/text()')
            i.add_value('id', n)

            # retornamos el item
            yield i.load_item()

In [ ]:
!scrapy runspider spider.py -o stack_preguntas.csv -t csv

## Crawler

In [1]:
from scrapy.item import Field, Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose

class CasaItem(Item):
    tipo = Field()
    capacidad = Field()
    precio = Field()


class CasaCrawler(CrawlSpider):
    name = "Crawler_01"
    start_urls = ['https://es-l.airbnb.com/s/homes?refinement_paths[]=homes/section/NEARBY_LISTINGS']
    allowed_domains = ['airbnb.com']  # Lista de dominios permitidos
    
    # Restricciones que debe acatar el crawler
    rules = (
        Rule(LinkExtractor(allow=r'items_offset=')),  # Puede ingresar a las páginas que tienen en la url la expresión regular
        Rule(LinkExtractor(allow=r'/rooms'), callback='parse_items'),
    )
    
    # Función que es llamada cuando se cumpla la regla
    def parse_items(self, response):
        item = ItemLoader(CasaItem(), response)
        item.add_xpath('tipo', '//*[@id="site-content"]/div/div[4]/div/div/div[1]/div[2]/div/div/div/section/div/div[1]/div[2]/div[1]/text()')
        item.add_xpath('capacidad', '//*[@id="site-content"]/div/div[4]/div/div/div[1]/div[1]/div/div/div/div/section/div/div/div/div[1]/div[2]/span[1]/text()')
        item.add_xpath('precio', '//*[@id="site-content"]/div/div[4]/div/div/div[3]/div/div/div[1]/div/div/div/div/div[1]/div[1]/div/div/span/span[1][@class="_pgfqnw"]/text()')
        
        yield item.load_item()